Edit -> Notebook Setting -> Hardware Accelerator -> Gpu

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import os

import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import pdb


if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')




In [ ]:
def to_img(x):
    x = 0.5 * (x + 1)
    x = x.clamp(0, 1)
    x = x.view(x.size(0), 1, 28, 28)
    return x


num_epochs = 50
batch_size = 128
learning_rate = 1e-3

img_transform = transforms.Compose([
    transforms.ToTensor()])



In [ ]:
dataset = MNIST('./data', transform=img_transform,download=True)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(True),
            nn.Linear(128, 64),
            nn.ReLU(True), nn.Linear(64, 12), nn.ReLU(True), nn.Linear(12, 3))
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.ReLU(True),
            nn.Linear(12, 64),
            nn.ReLU(True),
            nn.Linear(64, 128),
            nn.ReLU(True), nn.Linear(128, 28 * 28), nn.Sigmoid())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
    model.parameters(), lr=learning_rate, weight_decay=1e-5)



In [ ]:
for epoch in range(num_epochs):
    for data in dataloader:
        img, _ = data
        #pdb.set_trace()
        img = img.view(img.size(0), -1)
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    #print('epoch [{}/{}], loss:{:.4f}'
     #     .format(epoch + 1, num_epochs, loss.data[0]))
    if epoch % 10 == 0:
        print('epoch: '+str(epoch))
        pic = to_img(output.cpu().data)
        picOriginal = to_img(img.cpu().data)

        plt.subplot(121)
        plt.imshow(picOriginal[0,0,:,:])
        plt.title("Original Image")
        plt.subplot(122)
        plt.imshow(pic[0,0,:,:]) 
        plt.title("Reconstructed Image")       
        plt.show()
        
        #save_image(pic, './mlp_img/image_{}.png'.format(epoch))

#torch.save(model.state_dict(), './sim_autoencoder.pth')


In [ ]:
model_save_name='sim_autoencoder.pth'
path = F"/content/gdrive/My Drive/Colab Notebooks/{model_save_name}"

In [ ]:
torch.save(model.state_dict(), path)

In [ ]:
# divide dataset into training and validation datasets
# after training the network on tranining set, evaluate its performance on validation set 

In [ ]:
# plot training and validation loss for each epoch

In [ ]:
# denoising autoencoder
# which change you can to convert the network to denoising autoencoder
# Tip:  add noise to input during training but reconstruct the original dataset.
# use the example below to generate noise
#t = torch.Tensor(10,10).cuda()
#t.normal_()

In [ ]:
# use the output of the encoder for classification of digits
# use softmax as activation function
# use pretrained weights of the network as feature extractor
#model.load_state_dict (torch.load(path) ,strict=False)